In [ ]:
# Per evitare errori di caricamento dei moduli, eseguire il seguente comando in un terminale di Anaconda (o nel terminale, se si usa macOS)
# il comando va eseguito solo una volta, e non è necessario rieseguirlo in futuro:
# pip install bs4 lxml

# lettura di file JSON; utilizzo di BeautifulSoup; lavoro con i file XML
import os
import re
from glob import glob
import json
from bs4 import BeautifulSoup
from lxml import etree

In [ ]:
# Impostare manualmente il codice della lingua ISO 3166-1 a 2 lettere (ad esempio, English = en), in modo che
# vengano elaborati solo i file di sottotitoli per la lingua impostata.
language_code = "en"

# Crea un'espressione regolare per catturare il titolo del video incluse the sigle di denominazione predefinite di yt-dlp, dove:
# [FILENAME].info.json = il file JSON contenente i metadati dettagliati
# [FILENAME].[LL].srv3 = il file XML contenente i sottotitoli in formato SRV3, dove [LL] è il codice di 2 lettere della lingua ISO 3166-1
# [FILENAME].[LL].ttml = il file XML contenente i sottotitoli in formato TTML, dove [LL] è il codice lingua ISO 3166-1 di 2 lettere
filename_filter = re.compile(
    "(.*?)\.(info\.json|[A-Za-z]{1,3}\.srv3|[A-Za-z]{1,3}\.ttml)"
)
# Creare un elenco vuoto per memorizzare tutti i titoli dei video.
unique_filenames_list = []
# Elenca tutti i nomi dei file presenti nella cartella in cui risiede lo script
files = glob("*.*")
print(files)

In [ ]:
# Per ogni singolo nome di file trovato nella cartella, eseguire:
for single_file in files:
    # Cerca l'espressione regolare per catturare i metadati e i file dei sottotitoli nel nome del file e memorizza il risultato
    # nella variabile "found_filename".
    found_filename = re.search(filename_filter, single_file)
    # Se il nome del file corrisponde all'espressione regolare, estrae il nome del file senza le estensioni; quindi controlla se
    # il nome del file pulito è presente nell'elenco dei nomi di file e, in caso contrario, lo aggiunge.
    if found_filename is not None and found_filename[1] not in unique_filenames_list:
        unique_filenames_list.append(found_filename[1])


# Creare la funzione per convertire il formato temporale utilizzato da TTML (HH:MM:SS.MS) in quello utilizzato da SRV3
# (numero totale di millisecondi); adattato da
# https://stackoverflow.com/questions/59314323/how-to-convert-timestamp-into-milliseconds-in-python
def convert_timestamp(msf):
    hours, minutes, seconds = msf.split(":")
    seconds, milliseconds = seconds.split(".")
    hours, minutes, seconds, milliseconds = map(
        int, (hours, minutes, seconds, milliseconds)
    )
    return (hours * 3600 + minutes * 60 + seconds) * 1000 + milliseconds


# Per ogni nome di file unico trovato, eseguire:
for filename in unique_filenames_list:
    # Ricreare i nomi completi dei file con le estensioni e memorizzare ciascuno di essi in una singola variabile
    json_file = filename + ".info.json"
    srv3_file = filename + "." + language_code + ".srv3"
    ttml_file = filename + "." + language_code + ".ttml"
    # Creare l'elemento XML <text>, elemento principale dell'output finale.
    text_tag = etree.Element("text")

    # Aprire il file JSON dei metadati:
    metadata_file = json.loads(open(json_file, encoding="utf-8").read())
    # Aggiungere una serie di punti di metadati come attributi del tag <text>.
    text_tag.attrib["uploader"] = metadata_file["uploader"]
    text_tag.attrib["date"] = metadata_file["upload_date"]
    text_tag.attrib["views"] = str(metadata_file["view_count"])
    text_tag.attrib["title"] = metadata_file["fulltitle"]
    text_tag.attrib["livestreaming"] = str(metadata_file["was_live"])
    # Controllare se il punto di metadati 'like_count' è presente, in caso contrario assegnare il valore "na" all'attributo 'like_count'.
    text_tag.attrib["likes"] = str(
        metadata_file["like_count"] if "like_count" in metadata_file else "na"
    )

    # Controlla se il file SRV3 esiste (viene data priorità a SRV3 rispetto a TTML per la presenza di dettagli di autotrascrizione);
    # in caso affermativo, eseguire:
    if os.path.isfile(srv3_file):
        # Assegnare l'attributo 'format' con valore 'srv' al tag dell'elemento <text>.
        text_tag.attrib["format"] = "srv3"
        # Creare il nome del file di uscita utilizzando il nome del file di ingresso
        output_filename = srv3_file + ".xml"
        # Apri il file SRV3
        f = open(srv3_file, "r", encoding="utf-8")
        # Analizzare il suo contenuto XML usando BeautifulSoup
        soup = BeautifulSoup(f, features="xml")
        # Se nel tag dell'elemento <s> si trova l'attributo 'ac' (= autocaption), 
        # allora i sottotitoli sono il risultato dell'autocaptioning; quindi assegnare il valore
        # 'true' alla variabile 'is_ac'. Altrimenti, assegnare il valore 'false' a 'is_ac'.
        if soup.body.find_all("s", attrs={"ac": True}):
            is_ac = "true"
        else:
            is_ac = "false"

        # Assegnare il valore di 'is_ac' all'attributo 'autocaption' del tag dell'elemento <text>.
        text_tag.attrib["autocaption"] = is_ac

        # Per ogni paragrafo (cioè ogni riga dei sottotitoli) del file, eseguire:
        for sent in soup.body.find_all("p"):
            # Controlla se il contenuto testuale del paragrafo è più lungo di 1 carattere; questo evita di aggiungere paragrafi vuoti all'output finale.
            if len(sent.get_text()) > 1:
                # Creare un elemento <p> all'interno dell'output XML.
                p_tag = etree.SubElement(text_tag, "p")
                # Aggiungere l'attributo 'time' (che indica l'ora di inizio del paragrafo) e assegnargli il valore che compare in 't'.
                p_tag.attrib["time"] = str(sent["t"])
                # Aggiungere l'attributo 'is_ac' e assegnargli il valore della variabile 'is_ac' creata in precedenza.
                p_tag.attrib["is_ac"] = is_ac
                p_tag.text = sent.get_text()
            # Se il paragrafo non contiene testo (cioè la sua lunghezza è < 1), lo si salta.
            else:
                continue

    # Se il file SRV3 non esiste, controllare se esiste invece il file TTML; quindi eseguire
    # (solo i passaggii che differiscono dalla procedura SRV3 sono commentati):
    elif os.path.isfile(ttml_file):
        text_tag.attrib["format"] = "ttml"
        text_tag.attrib["autocaption"] = "na"
        output_filename = ttml_file + ".xml"
        f = open(ttml_file, "r", encoding="utf-8")
        soup = BeautifulSoup(f, features="xml")

        for sent in soup.body.find_all("p"):
            if len(sent.get_text()) > 1:
                p_tag = etree.SubElement(text_tag, "p")
                # Aggiungere l'attributo 'time', assegnandogli come valore l'ora di partenza dall'attributo 'begin' del file originale,
                # convertito in millisecondi con la funzione "convert_timestamp".
                p_tag.attrib["time"] = str(convert_timestamp(str(sent["begin"])))
                p_tag.attrib["is_ac"] = "na"
                p_tag.text = sent.get_text()
            else:
                continue
    # Se non vengono trovati né i file SRV3 né i file TTML, viene stampato "Nessun file di sottotitoli trovato".
    else:
        print("No subtitle files found.")

    # Scrivere i dati estratti formattati in XML nella struttura XML finale.
    tree = etree.ElementTree(text_tag)
    # Scrivere l'XML nel file di output
    tree.write(
        output_filename, pretty_print=True, xml_declaration=True, encoding="utf-8"
    )